In [1]:
import pandas as pd
import os 
from time import gmtime, strftime
import datetime
import xlrd  


In [2]:
xlsfile='testdata/102_Demo_Data.xlsx'
print (xlsfile)
df = pd.DataFrame([])

ts= strftime("%Y-%m-%d %H:%M:%S")

book = xlrd.open_workbook(xlsfile)
n= book.nsheets
print ("Number of sheets:", n)



testdata/102_Demo_Data.xlsx
Number of sheets: 3


In [3]:
# Checks number of columns and headers
for x in range(0, n):
    print (x,"/",n)
    print(book.sheet_names()[x])
    sh=book.sheet_by_index(x)
    num_cols = sh.ncols   # Number of columns
    for c in range (0,num_cols):
        print(str(c) +" "+ sh.cell_value(0,c))
        

0 / 3
Audits
0 Name
1 Title
2 Expenses
3 Date
1 / 3
AllHands
0 Name
1 Title
2 Expenses
3 Date
2 / 3
Training
0 Name
1 Title
2 Expenses
3 Date


In [4]:
input_frames=[]
for x in range(0, n):
    sh=book.sheet_by_index(x)
    cellA1= sh.cell_value(0,0)
    print (x,"/",n)
    print (cellA1)
    print ("Processing sheet " + book.sheet_names()[x])
    if cellA1=="Name":    
        num_cols = sh.ncols   # Number of columns
        num_rows = sh.nrows
        print ("Num Cols: ",num_cols, "Num Rows:", num_rows)
        data=[]
        for row_idx in range(0, num_rows):    # Iterate through rows use sh.nrows for all rows    
            row_data=[]
            for col_idx in range(0, num_cols):  # Iterate through columns
                
                
                
                cell_obj = sh.cell(row_idx, col_idx)  # Get cell object by row, col
                cell_value= cell_obj.value
                cell_type = sh.cell_type(row_idx, col_idx)
                
                if cell_type == xlrd.XL_CELL_DATE:
                    get_col= xlrd.xldate.xldate_as_datetime (cell_value, book.datemode)
                elif cell_type == xlrd.XL_CELL_NUMBER:
                    get_col = int(cell_value)
                else:
                    get_col = cell_value
               
                row_data.append(get_col)
            data.append(row_data)

        frame=pd.DataFrame(data[1:],columns=data[0])
        #We add here a column for the source sheet and a timestamp
        frame["SourceSheet"]=sh.name
        frame["Timestamp"]= ts
        input_frames.append (frame)

#Here we concatenate the multiple dataframes we created , one per sheet
# the ignore_index option is so that the main index is recreated so each row will have a unique ID 
if len(input_frames) >0:   
    df = pd.concat(input_frames, ignore_index=True)
    


0 / 3
Name
Processing sheet Audits
Num Cols:  4 Num Rows: 13
1 / 3
Name
Processing sheet AllHands
Num Cols:  4 Num Rows: 9
2 / 3
Name
Processing sheet Training
Num Cols:  4 Num Rows: 7


In [5]:
print (list(df))
df.rename(columns={'Timestamp':'Date Processed'}, inplace=True)
print ("Top 10 records")
df

['Name', 'Title', 'Expenses', 'Date', 'SourceSheet', 'Timestamp']
Top 10 records


,Name,Title,Expenses,Date,SourceSheet,Date Processed
0,John,Senior,1000,2015-08-15,Audits,2019-08-18 19:35:43
1,Peter,Manager,1500,2014-12-01,Audits,2019-08-18 19:35:43
2,Arne,Senior,-1,2015-07-01,Audits,2019-08-18 19:35:43
3,Hiten,Senior,,2015-01-01,Audits,2019-08-18 19:35:43
4,Najwan,Senior,830,2015-10-01,Audits,2019-08-18 19:35:43
5,Peter,Manager,3000,2015-11-30,Audits,2019-08-18 19:35:43
6,Liz,senior,200,2015-10-14,Audits,2019-08-18 19:35:43
7,Diana,senior,200,2015-10-14,Audits,2019-08-18 19:35:43
8,Diana,senior,199,2015-10-14,Audits,2019-08-18 19:35:43
9,John,senior,162,2015-01-11,Audits,2019-08-18 19:35:43


In [6]:
print ("Record count (non null):")
print(df.count())
print ("Max:",pd.to_numeric(df["Expenses"]).max())
print ("Min", pd.to_numeric(df["Expenses"]).min())


Record count (non null):
Name              26
Title             26
Expenses          26
Date              26
SourceSheet       26
Date Processed    26
dtype: int64
Max: 3000.0
Min -1.0


In [7]:
# This is an optional command to set the index, if we had some column that can act as index, otherwise, we get the default row number  on import
#df.set_index("Name", inplace=True, drop=False)


In [8]:
#Now we operate on the DF
dupli_df= df[df.duplicated(['Name','Date', 'Expenses'], keep=False)].sort_values(by=["Expenses"])
print (len(dupli_df))
dupli_df[0:100]

2


,Name,Title,Expenses,Date,SourceSheet,Date Processed
17,Peter,Manager,2000,2015-08-31,AllHands,2019-08-18 19:35:43
25,Peter,Manager,2000,2015-08-31,Training,2019-08-18 19:35:43


In [9]:
# Create a Pandas Excel writer using XlsxWriter as the engine.

if os.path.exists('output/102_Output_Duplicates.xlsx'):
    try:
        os.remove('testdata/102_Output_Duplicates.xlsx')
        print("Deleted previous file")
        #you could decide here to rename previous extractions, or 
        #adding a timestamp to output file and cmment out this step  
    except:
        print("Some  error trying to access/delete the previous file")
        quit()
     
try:
    writer = pd.ExcelWriter('output/102_Output_duplicates.xlsx', engine='xlsxwriter')
    dupli_df.to_excel(writer, sheet_name='Duplicates')
    df[0:10].to_excel(writer, sheet_name="TopTen")
    writer.save()
    print("Saved new output file")
       
except:
   print ("error creating/saving excel file")
 
    

Saved new output file
